In [ ]:
from astropy.io import fits
import astropy.constants as c
import astropy.units as u
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re
import os
from scipy.signal import find_peaks

In [ ]:
import importlib
import util
importlib.reload(util) # run this each time util is changed, to prevent restarting kernel

## Load data

In [ ]:
dirname = '20240504'

In [ ]:
fpaths = glob.glob(f'{dirname}/*.fits')
print(fpaths)

In [ ]:
with fits.open('20240504/red0018.fits') as hdul:
    hdul_r18 = hdul
hdul_r18[0].header

In [ ]:
with fits.open('20240504/blue0018.fits') as hdul:
    hdul_b18 = hdul
hdul_b18[0].header

In [ ]:
hdr_keys = ['OBJECT', 'IMGTYPE', 'GAIN', 'EXPTIME', 'UT', 'RA', 'DEC', 'AIRMASS', 'GRATING']

In [ ]:
hdr_data = []
data = {}
for fpath in fpaths:
    with fits.open(fpath) as hdul:
        k = re.split(r'/|\.', fpath)[1]
        hdr = hdul[0].header
        hdr_data.append([hdr.get(k, None)  for k in hdr_keys])
        data[k] = hdul[0].data.astype(float)

In [ ]:
len(data)

In [ ]:
hdr_df = pd.DataFrame(hdr_data, columns=hdr_keys, index=[re.split(r'/|\.', fpath)[1] for fpath in fpaths])
hdr_df.index.name = 'filename'
hdr_df.sort_index()

In [ ]:
plt.figure(figsize=(15,3))
plt.imshow(data['blue0023'].T, origin='lower', clim=np.percentile(data['blue0023'].T, (1,99)))
plt.colorbar()

In [ ]:
util.plot_frame(data['blue0023'].T, label='counts', title='Blue 0023 SN2024eze', prange=(5,95))

In [ ]:
util.plot_frame(data['blue0014'].T, label='counts', title='Blue 0014 Arc', prange=(5,95))

In [ ]:
util.plot_frame(data['blue0004'].T, label='counts', title='Blue 0004 Bias', prange=(5,95))

In [ ]:
util.plot_frame(data['blue0028'].T, label='counts', title='Blue 0028 Calib star', prange=(5,95))

In [ ]:
util.plot_frame(data['red0023'], label='counts', title='Red 0023 SN2024eze', prange=(10, 90))

In [ ]:
util.plot_frame(data['red0020'], label='counts', title='Red 0020 Flat', prange=(10, 90))

In [ ]:
util.plot_frame(data['red0014'], label='counts', title='Red 0014 Arc', prange=(10, 90))

In [ ]:
util.plot_frame(data['red0027'], label='counts', title='Red 0027 Calib star', prange=(10, 90))

## Prepare master frames
- for flat, bias, arc (denoted as cal in IMGTYPE), calib star

In [ ]:
red_shape = data['red0001'].shape
blue_shape = data['blue0001'].shape

red_shape,blue_shape

In [ ]:
bias = hdr_df[hdr_df['IMGTYPE'] == 'bias']
flats = hdr_df[hdr_df['IMGTYPE'] == 'flat']
arcs = hdr_df[hdr_df['IMGTYPE'] == 'cal']
calib_star = hdr_df[(hdr_df['IMGTYPE'] == 'object') & (hdr_df['OBJECT'] == 'HD 158261')]

In [ ]:
#remove saturated frames for blue calib star
calib_star.drop(calib_star.loc[(calib_star.index.str.contains('blue')) & (calib_star['EXPTIME'] > 0.5)].index, inplace=True)
calib_star

### Master bias (red)

In [ ]:
def create_master(frame, color, bias=None):
    frame_filtered = frame[np.char.startswith(list(frame.index), color)]
    shape = red_shape if color=='red' else blue_shape
    layers = np.zeros((shape[0], shape[1], frame_filtered.shape[0]))
    for i, curr_file in enumerate(frame_filtered.index):
        curr_data = data[curr_file]
        if bias is not None:
            texp = frame_filtered.loc[curr_file, 'EXPTIME']
            curr_data = (curr_data - bias) / texp
        layers[:, :, i] = curr_data

    master = np.median(layers, axis=2)
    return master


In [ ]:
master_bias_red = create_master(bias, 'red')
util.plot_frame(master_bias_red, label='counts', title='Master Bias (red)', prange=(10, 90))

### Master bias (blue)

In [ ]:
master_bias_blue = create_master(bias, 'blue')
util.plot_frame(master_bias_blue.T, label='counts', title='Master Bias (blue)', prange=(10, 90))

### Master flat (red)

In [ ]:
master_flat_red = create_master(flats, 'red', master_bias_red)
util.plot_frame(master_flat_red, label='counts/s', title='Master flat (red)', prange=(5, 95))

### Master flat (blue)

In [ ]:
master_flat_blue= create_master(flats, 'blue', master_bias_blue)
util.plot_frame(master_flat_blue.T, label='counts/s', title='Master flat (blue)', prange=(5, 95))

### Master arc (red)

In [ ]:
master_arc_red = create_master(arcs, 'red', master_bias_red)
util.plot_frame(master_arc_red, label='counts/s', title='Master arc (red)', prange=(5, 95))

### Master arc (blue)

In [ ]:
master_arc_blue = create_master(arcs, 'blue', master_bias_blue)
util.plot_frame(master_arc_blue.T, label='counts/s', title='Master arc (blue)', prange=(5, 95))

## Rectify frames

### Rectify blue
#### Find trace using calib star

In [ ]:
master_calib_star_blue = create_master(calib_star, 'blue', master_bias_blue).T

In [ ]:
x_edges_B = [100, 1200, 2400] #0 is dead
y_edges_B = [np.argmax(master_calib_star_blue[:,i]) for i in x_edges_B]

prange_B = (5,95)
util.plot_frame(master_calib_star_blue, figsize=(15, 5), aspect=5, label='counts/s', title='Blue Calib star', prange=prange_B)
plt.plot(x_edges_B, y_edges_B, label='trace', color='r')
plt.ylim(250, 350)

In [ ]:
trace_x_B = np.arange(0, master_calib_star_blue.shape[1], 1) # x indices remain same
# fit a polynomial using x,y edges_B and evaluate entire trace
trace_y_B = np.round(np.polyval(np.polyfit(x_edges_B, y_edges_B, 1), trace_x_B)).astype(int) #y indices has to be integers
trace_x_B.shape, trace_y_B.shape

In [ ]:
util.plot_frame(master_calib_star_blue, figsize=(15, 5), aspect=5, label='counts/s', title='Blue Calib star', prange=prange_B)
plt.plot(trace_x_B, trace_y_B, label='trace', color='r')
plt.ylim(250, 350)

#### Find y pixel bounds using flat

In [ ]:
util.plot_frame(master_flat_blue.T, aspect=2, label='counts/s', title='Blue Flat', prange=prange_B)
plt.plot(trace_x_B, trace_y_B, color='r')

y_bound_upper_B, y_bound_lower_B = (91, 200)
plt.plot(trace_x_B, trace_y_B+y_bound_upper_B, color='b')
plt.plot(trace_x_B, trace_y_B-y_bound_lower_B, color='b')

In [ ]:
master_flat_blue.T.shape

In [ ]:
# 1. Transpose array [where needed]
# 2. Pick values column wise within the y bounds
rectified_flat_B, rectified_calib_B, rectified_bias_B, rectified_arc_B  = [
    util.rectify_frame(frame, trace_y_B, y_bound_upper_B, y_bound_lower_B) for frame in
    [master_flat_blue.T, master_calib_star_blue, master_bias_blue.T, master_arc_blue.T]]
rectified_flat_B.shape

#### Find X-trim bounds
Only bright area of flat has meaningful data (division by normalised flat suffers if x isn't trimmed)

In [ ]:
flat1D_B = np.median(rectified_flat_B, axis=0)
x_trim_mask_B = np.where((flat1D_B >= np.max(flat1D_B) * 0.03))[0] # selecting 0.03 based on 1D flat

plt.figure(figsize=(12,4))
plt.plot(flat1D_B)
plt.axvline(x_trim_mask_B[0], color='r', alpha=0.5)
plt.axvline(x_trim_mask_B[-1], color='r', alpha=0.5)
plt.axhline(0, color='black', alpha=0.3)

In [ ]:
# 3. pick values within x-bounds
# 4. flip along X-axis (because blue has inverted wavelength axis)
rectified_flat_B, rectified_calib_B, rectified_bias_B, rectified_arc_B  = [
    frame[:, x_trim_mask_B][:,::-1] for frame in
    [rectified_flat_B, rectified_calib_B, rectified_bias_B, rectified_arc_B]]
rectified_flat_B.shape

#### Rectified blue frames plots

In [ ]:
trace_y_idx_B = y_bound_lower_B

In [ ]:
util.plot_frame(rectified_calib_B, label='counts/s', title='Rectified Blue Calib star', prange=prange_B)
plt.axhline(trace_y_idx_B, color='r', alpha=0.9, linewidth=0.5)

In [ ]:
util.plot_frame(rectified_flat_B, label='counts/s', title='Rectified Blue Flat', prange=prange_B)

In [ ]:
util.plot_frame(rectified_bias_B, label='counts', title='Rectified Blue Bias', prange=prange_B)


In [ ]:
util.plot_frame(rectified_arc_B, label='counts/s', title='Rectified Blue Arc', prange=prange_B)

### Rectify Red
#### Find trace

In [ ]:
master_calib_star_red = create_master(calib_star, 'red', master_bias_red)

In [ ]:
x_edges_R = [400, 3600, 4100]
y_edges_R = [np.argmax(master_calib_star_red[:,i]) for i in x_edges_R]

prange_R = (7,93)
util.plot_frame(master_calib_star_red, figsize=(15, 5), aspect=2.5, label='counts/s', title='Red Calib star', prange=prange_R)
plt.plot(x_edges_R, y_edges_R, label='trace', color='r')


In [ ]:
trace_x_R = np.arange(0, master_calib_star_red.shape[1], 1) # x indices remain same
trace_y_R = np.round(np.polyval(np.polyfit(x_edges_R, y_edges_R, 1), trace_x_R)).astype(int) #y indices has to be integers
trace_x_R.shape, trace_y_R.shape

#### Find y pixel bounds

In [ ]:
util.plot_frame(master_flat_red, figsize=(15, 5), aspect=2.5, label='counts/s', title='Red Calib star', prange=prange_R)
plt.plot(trace_x_R, trace_y_R, color='r')

y_bound_upper_R, y_bound_lower_R = (282, 120)
plt.plot(trace_x_R, trace_y_R+y_bound_upper_R, color='b')
plt.plot(trace_x_R, trace_y_R-y_bound_lower_R, color='b')

In [ ]:
master_flat_red.shape

In [ ]:
# 1. Pick values column wise within the y bounds
rectified_flat_R, rectified_calib_R, rectified_bias_R, rectified_arc_R  = [
    util.rectify_frame(frame, trace_y_R, y_bound_upper_R, y_bound_lower_R) for frame in
    [master_flat_red, master_calib_star_red, master_bias_red, master_arc_red]]
rectified_flat_R.shape

#### Find X-trim bounds
Need to remove dead area of detector and include only bright area of flat has meaningful data (division by normalised flat suffers if x isn't trimmed)

In [ ]:
util.plot_frame(rectified_calib_R, label='counts/s', title='Rectified Red Calib star', prange=prange_R)
plt.axvline(740, color='r', linestyle='--') # need to discard the bad section too

In [ ]:
np.median(rectified_calib_R, axis=0)[720: 750]

In [ ]:
np.median(rectified_calib_R, axis=0)[731:733]

In [ ]:
flat1D_R = np.median(rectified_flat_R, axis=0)

x_trim_mask_R = np.where((flat1D_R >= np.max(flat1D_R) * 0.03))[0]

plt.figure(figsize=(12,4))
plt.plot(flat1D_R)
plt.axvline(x_trim_mask_R[0], color='r', alpha=0.5)
plt.axvline(x_trim_mask_R[-1], color='r', alpha=0.5)
plt.axvline(732, color='r', linestyle='--', alpha=0.5) # the bad section
plt.axhline(0, color='black', alpha=0.3)

In [ ]:
x_trim_mask_R = x_trim_mask_R[x_trim_mask_R>=732]
x_trim_mask_R

In [ ]:
rectified_flat_R

In [ ]:
# 2. pick values within x-bounds
rectified_flat_R, rectified_calib_R, rectified_bias_R, rectified_arc_R  = [
    frame[:, x_trim_mask_R] for frame in
    [rectified_flat_R, rectified_calib_R, rectified_bias_R, rectified_arc_R]]
rectified_flat_R.shape

#### Rectifed Red frames plots

In [ ]:
trace_y_idx_R = y_bound_lower_R

In [ ]:
util.plot_frame(rectified_calib_R, label='counts/s', title='Rectified Red Calib star', prange=prange_R)
plt.axhline(trace_y_idx_R, color='r', alpha=0.7)

In [ ]:
util.plot_frame(rectified_flat_R, label='counts/s', title='Rectified Red Flat', prange=prange_R)

In [ ]:
util.plot_frame(rectified_bias_R, label='counts', title='Rectified Red Bias', prange=prange_R)

In [ ]:
util.plot_frame(rectified_arc_R, label='counts/s', title='Rectified Red Arc', prange=prange_R)

## Reduce frames 
- subtract bias, divide by exptime - already done by create_master()
- divide by normalised flat

In [ ]:
rectified_flat_B.shape, rectified_flat_R.shape

### Red

In [ ]:
normalised_flat_R = rectified_flat_R / np.max(rectified_flat_R)
util.plot_frame(normalised_flat_R, label='normalised counts/s', title='Normalised Red Flat', prange=prange_R)

In [ ]:
reduced_rect_calib_R = rectified_calib_R / normalised_flat_R

util.plot_frame(reduced_rect_calib_R, label='counts/s', title='Reduced, Rectified Red Calib star', prange=(2,98))
plt.axhline(trace_y_idx_R, color='r', alpha=0.7, linewidth=0.5)

### Blue

In [ ]:
normalised_flat_B = rectified_flat_B / np.max(rectified_flat_B)
util.plot_frame(normalised_flat_B, label='normalised counts/s', title='Normalised Blue Flat', prange=prange_B)

In [ ]:
reduced_rect_calib_B = rectified_calib_B / normalised_flat_B

util.plot_frame(reduced_rect_calib_B, label='counts/s', title='Reduced, Rectified Blue Calib star', prange=(2, 98))
plt.axhline(trace_y_idx_B, color='r', alpha=0.7, linewidth=0.5)

## Wavelength calibration
- use arc to derive a px to wvl solution

### Blue

In [ ]:
arc_aperture_B = (rectified_calib_B.shape[0]-1 - trace_y_idx_B)
arc_aperture_B

In [ ]:
# take median and flip along x-axis
arc_1D_spectrum_B = np.median(rectified_arc_B[trace_y_idx_B - arc_aperture_B : trace_y_idx_B + arc_aperture_B + 1], axis=0)
rectified_arc_B.shape, arc_1D_spectrum_B.shape

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(arc_1D_spectrum_B)
plt.ylabel('median counts/s')
plt.title('Blue Arc spectrum')
plt.xlim(0, 1200)
plt.ylim(-5, np.max(arc_1D_spectrum_B)+15) # to allow space for annotations

peaks_x_B, _ = find_peaks(arc_1D_spectrum_B, height=8, distance=12)

for i, px in enumerate(peaks_x_B):
    plt.axvline(px, color='r', alpha=0.75, linewidth=0.5)
    plt.text(px+4, np.max(arc_1D_spectrum_B)+12-(5*(i%3)), f'{px}', fontsize=8, ha='center', va='center')

In [ ]:
# Identified peaks from figure 10c, 10b (page 29-30 in https://sites.astro.caltech.edu/palomar/observer/200inchResources/dbsp/dbspArcAtlas.pdf)
# peaks_wvl_B = {
#     1816: (4764.865, 'Ar'), # tallest in 10c
#     # L to R to tallest ----
#     1854: (4806.021, 'Ar'),
#     1893: (4847.810, 'Ar'),
#     1922: (4879.864, 'Ar'),
#     # R to L of tallest -----
#     1782: (4726.868, 'Ar'),
#     1718: (4657.901, 'Ar'),
#     1674: (4609.567, 'Ar'),
#     1615: (4545.052, 'Ar'),
#     1557: (4481.811, 'Ar'),
#     1506: (4426.001, 'Ar'),
#     1434: (4348.064, 'Ar'),
#     1369: (4277.528, 'Ar'),
#     1352: (4259.362, 'Ar'),
#     1298: (4198.317, 'Ar'),
#     1259: (4158.591, 'Ar'),
#     # more in 10b -------
#     1234: (4131.724, 'Ar'),
#     1209: (4103.912, 'Ar'),
#     1155: (4045.813, 'Fe'),
#     1007: (3886.282, 'Fe'),
#     983: (3859.911, 'Fe'),
#     895: (3763.789, 'Fe'),
#     853: (3719.935, 'Fe'),
#     723: (3581.193, 'Fe')
# }

# using: {key - 910: value for key, value in peaks_wvl_B.items() if key - 910 >= 0}
peaks_wvl_B = {
 906: (4764.865, 'Ar'),
 944: (4806.021, 'Ar'),
 983: (4847.81, 'Ar'),
 1012: (4879.864, 'Ar'),
 872: (4726.868, 'Ar'),
 808: (4657.901, 'Ar'),
 764: (4609.567, 'Ar'),
 705: (4545.052, 'Ar'),
 647: (4481.811, 'Ar'),
 596: (4426.001, 'Ar'),
 524: (4348.064, 'Ar'),
 459: (4277.528, 'Ar'),
 442: (4259.362, 'Ar'),
 388: (4198.317, 'Ar'),
 349: (4158.591, 'Ar'),
 324: (4131.724, 'Ar'),
 299: (4103.912, 'Ar'),
 245: (4045.813, 'Fe'),
 97: (3886.282, 'Fe'),
 73: (3859.911, 'Fe')
 }

In [ ]:
plt.figure(figsize=(15, 3))
plt.plot(arc_1D_spectrum_B)
plt.ylabel('median counts/s')
plt.title('Blue Arc spectrum')
plt.xlim(0, 1200)
plt.ylim(-5, np.max(arc_1D_spectrum_B)+15) # to allow space for annotations


for i, (px, wvl) in enumerate(peaks_wvl_B.items()):
    line_clr = 'green' if wvl[1]=='Fe' else 'red'
    plt.axvline(px, color=line_clr, alpha=0.75, linewidth=0.5, label=wvl[1])
    plt.text(px+4, np.max(arc_1D_spectrum_B)+12-(5*(i%3)), f'{wvl[0]}', fontsize=8, ha='center', va='center')

# plt.legend();

In [ ]:
peaks_pixels_B = np.array(list(peaks_wvl_B.keys()))
peaks_wavelengths_B = np.array([wvl for wvl, _ in peaks_wvl_B.values()])

def wvl_solution_B(pixels, deg=2):
    coefficients = np.polyfit(peaks_pixels_B, peaks_wavelengths_B, deg)
    return np.polyval(coefficients, pixels)

plt.scatter(peaks_pixels_B, peaks_wavelengths_B, label='blue arc')
plt.plot(peaks_pixels_B, wvl_solution_B(peaks_pixels_B), color='r', label='wavelength solution')
plt.xlabel('pixels')
plt.ylabel('wavelengths [A]')
plt.title('Peaks')
plt.legend();

### Red

In [ ]:
arc_aperture_R = trace_y_idx_R
arc_aperture_R

In [ ]:
arc_1D_spectrum_R = np.median(rectified_arc_R[trace_y_idx_R - arc_aperture_R : trace_y_idx_R + arc_aperture_R + 1], axis=0)
arc_1D_spectrum_R.shape

In [ ]:
plt.figure(figsize=(15, 4))
plt.plot(arc_1D_spectrum_R)
plt.ylabel('median counts/s')
plt.title('Red Arc spectrum')
plt.xlim(100, 2800)
plt.ylim(-500, np.max(arc_1D_spectrum_R)+5000) # to allow space for annotations

peaks_x_R, _ = find_peaks(arc_1D_spectrum_R, height=1000, distance=12)

for i, px in enumerate(peaks_x_R):
    plt.axvline(px, color='r', alpha=0.75, linewidth=0.5)
    plt.text(px+4, np.max(arc_1D_spectrum_R)+(1900*(i%3)), f'{px}', fontsize=8, ha='center', va='center')
peaks_x_R

In [ ]:
# Identified peaks from figure 16b, 18a (https://sites.astro.caltech.edu/palomar/observer/200inchResources/dbsp/dbspArcAtlas.pdf)
# peaks_wvl_R = {
#     3385: (9657.786, 'Ar'),
#     3105: (9224.499, 'Ar'),
#     3040: (9122.967, 'Ar'),
#     2651: (8521.442, 'Ar'),
#     2589: (8424.648, 'Ar'),
#     2485: (8264.522, 'Ar'),
#     2388: (8115.311, 'Ar'),
#     2323: (8014.786, 'Ar'),
#     2280: (7948.176, 'Ar'),
#     2135: (7723.761, 'Ar'),
#     2077: (7635.106, 'Ar'),
#     1992: (7503.869, 'Ar'),
#     1231: (6334.428, 'Ne'),  # much less confident in Ne matches
#     1263: (6382.992, 'Ne'),
#     1276: (6402.246, 'Ne'),
#     1344: (6506.628, 'Ne'),
#     1456: (6678.276, 'Ne'),
#     1481: (6717.043, 'Ne'),
# }

# {key - 733: value for key, value in peaks_wvl_R.items() if key - 733 >= 0}

peaks_wvl_R = {
 2652: (9657.786, 'Ar'),
 2372: (9224.499, 'Ar'),
 2307: (9122.967, 'Ar'),
 1918: (8521.442, 'Ar'),
 1856: (8424.648, 'Ar'),
 1752: (8264.522, 'Ar'),
 1655: (8115.311, 'Ar'),
 1590: (8014.786, 'Ar'),
 1547: (7948.176, 'Ar'),
 1402: (7723.761, 'Ar'),
 1344: (7635.106, 'Ar'),
 1259: (7503.869, 'Ar'),
 498: (6334.428, 'Ne'),
 530: (6382.992, 'Ne'),
 543: (6402.246, 'Ne'),
 611: (6506.628, 'Ne'),
 723: (6678.276, 'Ne'),
 748: (6717.043, 'Ne')
}

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(arc_1D_spectrum_R)
plt.ylabel('median counts/s')
plt.title('Blue Arc spectrum')
plt.xlim(100, 2800)
plt.ylim(-500, np.max(arc_1D_spectrum_R)+5000) # to allow space for annotations


for i, (px, wvl) in enumerate(peaks_wvl_R.items()):
    line_clr = 'green' if wvl[1]=='Ne' else 'red'
    plt.axvline(px, color=line_clr, alpha=0.75, linewidth=0.5, label=wvl[1])
    plt.text(px+4, np.max(arc_1D_spectrum_R)+(1900*(i%3)), f'{wvl[0]}', fontsize=8, ha='center', va='center')

# plt.legend();

In [ ]:
peaks_pixels_R = np.array(list(peaks_wvl_R.keys()))
peaks_wavelengths_R = np.array([wvl for wvl, _ in peaks_wvl_R.values()])

def wvl_solution_R(pixels, deg=2):
    coefficients = np.polyfit(peaks_pixels_R, peaks_wavelengths_R, deg)
    return np.polyval(coefficients, pixels)

plt.scatter(peaks_pixels_R, peaks_wavelengths_R, label='red arc')
plt.plot(peaks_pixels_R, wvl_solution_R(peaks_pixels_R), color='r', label='wavelength solution')
plt.xlabel('pixels')
plt.ylabel('wavelengths [A]')
plt.title('Peaks')
plt.legend();

### Wavelength ranges

In [ ]:
rectified_arc_R.shape

In [ ]:
wvl_range_R = wvl_solution_R(np.arange(0, rectified_arc_R.shape[1]))
wvl_range_R

In [ ]:
rectified_arc_B.shape

In [ ]:
wvl_range_B = wvl_solution_B(np.arange(0, rectified_arc_B.shape[1]))
wvl_range_B

In [ ]:
# where R starts
wvl_range_R[0] 

In [ ]:
# find index of a number in B that is just smaller than R's start
just_before_R_idx = np.searchsorted(wvl_range_B, wvl_range_R[0])-1 

# right-truncated B wavlengths
end_idx_B = just_before_R_idx + 1 # because np slicing excludes end index
wvl_range_B_trunc = wvl_range_B[: end_idx_B]
wvl_range_B_trunc

## Airmass correction 
- use airmass extinction to find zero airmass flux for calib star 

## Flux calibration
- Source and BG extraction on calib star
- Derive counts to photons conversion factor for both red and blue

### source and background extraction

#### Red

In [ ]:
util.plot_frame(reduced_rect_calib_R, prange=(1,99), aspect=7)

aperture_idx_R = (trace_y_idx_R-1 - 6, trace_y_idx_R-1 + 6)
plt.axhline(y=aperture_idx_R[0], color='r', alpha=0.5)
plt.axhline(y=aperture_idx_R[1], color='r', alpha=0.5)
# plt.xlim(1000, 1200)
plt.ylim(trace_y_idx_R - 10 * 6,trace_y_idx_R + 10 * 6 ); 

In [ ]:
calib_spectrum_R = np.sum(
    rectified_calib_R[aperture_idx_R[0]: aperture_idx_R[1]+1, :],
    axis=0)
plt.plot(wvl_range_R, calib_spectrum_R)
plt.xlabel("wavelength (angstroms)")
plt.ylabel("Total counts / s")
plt.title("rectified calib star spectrum (RED)")

In [ ]:
calib_spectrum_R = np.sum(
    reduced_rect_calib_R[aperture_idx_R[0]: aperture_idx_R[1]+1, :],
    axis=0)
plt.plot(wvl_range_R, calib_spectrum_R)
plt.xlabel("wavelength (angstroms)")
plt.ylabel("Total counts / s")
plt.title("reduced rectified calib star spectrum (RED)")

In [ ]:
bg_upper_idx_R = (aperture_idx_R[1]+10, aperture_idx_R[1]+20)
bg_lower_idx_R = (aperture_idx_R[0]-20, aperture_idx_R[0]-10)

util.plot_frame(reduced_rect_calib_R, prange=(3,97), aspect=7)
plt.ylim(trace_y_idx_R - 10 * 6,trace_y_idx_R + 10 * 6 )
plt.axhline(y=aperture_idx_R[0], color='r')
plt.axhline(y=aperture_idx_R[1], color='r')

plt.axhline(bg_lower_idx_R[0], color='g', alpha=0.8)
plt.axhline(bg_lower_idx_R[1], color='g', alpha=0.8)
plt.axhline(bg_upper_idx_R[0], color='y', alpha=0.8)
plt.axhline(bg_upper_idx_R[1], color='y', alpha=0.8)

In [ ]:
upper_bg_R = reduced_rect_calib_R[bg_upper_idx_R[0]: bg_upper_idx_R[1]+1]
lower_bg_R = reduced_rect_calib_R[bg_lower_idx_R[0]: bg_lower_idx_R[1]+1]

bg_R = np.vstack([upper_bg_R, lower_bg_R])

bg_signal_R = np.median(bg_R, axis=0)
plt.plot(wvl_range_R, bg_signal_R)
plt.xlabel("wavelength (angstroms)")
plt.ylabel("Median counts / s")
plt.title("Red calib star background signal")

In [ ]:
aperture_nrows_R = aperture_idx_R[1]-aperture_idx_R[0]+1 #both are inclusive
summed_bg_R = bg_signal_R * aperture_nrows_R
calib_bg_subtracted_R = calib_spectrum_R - summed_bg_R
plt.plot(wvl_range_R, calib_bg_subtracted_R)

plt.xlabel("wavelength (angstroms)")
plt.ylabel("Total counts / s")
plt.title("reduced rectified background-subtracted calib star spectrum (RED)")

#### Blue

In [ ]:
util.plot_frame(reduced_rect_calib_B, prange=(1,99), aspect=5)

aperture_idx_B = (trace_y_idx_B+1 - 8, trace_y_idx_B+1 + 8)
plt.axhline(y=aperture_idx_B[0], color='r', alpha=0.5)
plt.axhline(y=aperture_idx_B[1], color='r', alpha=0.5)
# plt.xlim(1000, 1200)
plt.ylim(trace_y_idx_B - 60, trace_y_idx_B + 60); 

In [ ]:
calib_spectrum_B = np.sum(
    rectified_calib_B[aperture_idx_B[0]: aperture_idx_B[1]+1, :end_idx_B], 
    axis=0)
plt.plot(wvl_range_B_trunc, calib_spectrum_B)
plt.xlabel("wavelength (angstroms)")
plt.ylabel("Total counts / s")
plt.title("rectified calib star spectrum (BLUE)")

In [ ]:
calib_spectrum_B = np.sum(
    reduced_rect_calib_B[aperture_idx_B[0]: aperture_idx_B[1]+1, :end_idx_B], 
    axis=0)
plt.plot(wvl_range_B_trunc, calib_spectrum_B)
plt.xlabel("wavelength (angstroms)")
plt.ylabel("Total counts / s")
plt.title("reduced rectified calib star spectrum (BLUE)")

In [ ]:
bg_upper_idx_B = (aperture_idx_B[1]+10, aperture_idx_B[1]+20)
bg_lower_idx_B = (aperture_idx_B[0]-20, aperture_idx_B[0]-10)

util.plot_frame(reduced_rect_calib_B, prange=(3,97), aspect=5)
plt.ylim(trace_y_idx_B - 60, trace_y_idx_B + 60)
plt.axhline(y=aperture_idx_B[0], color='r')
plt.axhline(y=aperture_idx_B[1], color='r')

plt.axhline(bg_lower_idx_B[0], color='g', alpha=0.8)
plt.axhline(bg_lower_idx_B[1], color='g', alpha=0.8)
plt.axhline(bg_upper_idx_B[0], color='y', alpha=0.8)
plt.axhline(bg_upper_idx_B[1], color='y', alpha=0.8)

In [ ]:
lower_bg_B = reduced_rect_calib_B[bg_lower_idx_B[0] : bg_lower_idx_B[1]+1]
upper_bg_B = reduced_rect_calib_B[bg_upper_idx_B[0] : bg_upper_idx_B[1]+1]

bg_B = np.vstack([upper_bg_B, lower_bg_B])

bg_signal_B = np.median(bg_B[:, :end_idx_B], axis=0)
plt.plot(wvl_range_B_trunc, bg_signal_B)
plt.xlabel("wavelength (angstroms)")
plt.ylabel("Median counts / s")
plt.title("median background signal (BLUE)")

In [ ]:
aperture_nrows_B = aperture_idx_B[1]-aperture_idx_B[0]+1 #both are inclusive
summed_bg_B = bg_signal_B * aperture_nrows_B
calib_bg_subtracted_B = calib_spectrum_B - summed_bg_B
plt.plot(wvl_range_B_trunc, calib_bg_subtracted_B)
plt.xlabel("wavelength (angstroms)")
plt.ylabel("Total counts / s")
plt.title("reduced rectified background-subtracted calib star spectrum (blue)");

In [ ]:
wvl_solution = np.hstack([wvl_range_B_trunc, wvl_range_R])
calib_final = np.hstack([calib_bg_subtracted_B, calib_bg_subtracted_R])
plt.plot(wvl_solution, calib_final)
plt.xlabel("wavelength (angstroms)")
plt.ylabel("counts / s")
plt.title("reduced rectified background-subtracted calib star spectrum (COMBINED)");

### Reference spectrum from pysynphot

In [ ]:
# add env variable before importing pysynphot
pysyn_env_location = os.path.join(os.getcwd(), 'grp/redcat/trds/')
os.environ['PYSYN_CDBS'] = pysyn_env_location #note: CDBS not CBDS which is why it was throwing errors

import pysynphot as S

In [ ]:
# from https://articles.adsabs.harvard.edu/full/2006MNRAS.368..247C/0000247.000.html: 
# Teff: 9900+-220, log g: 4.00+-0.10, assuming metallicity [M/H] = 0

ref_spec = S.Icat('k93models', 9900, 0, 4.0)
ref_spec

In [ ]:
# resample ref spectrum to our wavelength grid
ref_spec_resampled = ref_spec.resample(wvl_solution) #TODO: try scipy.signal.resample() over this
ref_spec.wave.shape, ref_spec_resampled.wave.shape

In [ ]:
plt.plot(ref_spec_resampled.wave, ref_spec_resampled.flux)
# plt.xlim(wvl_solution[0],wvl_solution[-1])
plt.xlabel(f'Wavelength [{ref_spec_resampled.waveunits}]')
plt.ylabel(f'Flux [{ref_spec_resampled.fluxunits}]')
plt.title('Surface flux of Reference star (HD 158261)\n[Resampled to our wavelength grid]');

In [ ]:
# surface flux to absolute flux can be obtained by renormalisation
# as mentioned at 2nd para in https://pysynphot.readthedocs.io/en/latest/appendixa.html#kurucz-atlas


# from https://simbad.cds.unistra.fr/simbad/sim-id?Ident=HD+158261&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id
# V = 5.94 vegamag (default mag system is vega unless specified as per https://simbad.cds.unistra.fr/guide/sim-id.htx)
ref_spec_resampled_abs = ref_spec_resampled.renorm(5.94, 'vegamag', S.ObsBandpass('johnson,v'))

In [ ]:
plt.plot(ref_spec_resampled_abs.wave, ref_spec_resampled_abs.flux)
# plt.xlim(wvl_solution[0],wvl_solution[-1])
plt.xlabel(f'Wavelength [{ref_spec_resampled_abs.waveunits}]')
plt.ylabel(f'Flux [{ref_spec_resampled_abs.fluxunits}]')
plt.title('Absolute flux of Reference star (HD 158261)');

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(wvl_solution, calib_final/np.max(calib_final), label='observed')
plt.plot(ref_spec_resampled_abs.wave, ref_spec_resampled_abs.flux/np.max(ref_spec_resampled_abs.flux), label='reference', alpha=0.7)
plt.legend()
plt.xlabel('Wavelength [A]')
plt.ylabel('Normalised flux');
# plt.xlim(3700, 5000)

In [ ]:
ref_spec_resampled_abs.fluxunits # flam = erg s-1 cm-2 A-1

In [ ]:
calibration_flux = (ref_spec_resampled_abs.flux * u.Unit('erg s-1 cm-2 AA-1')) / (calib_final * u.Unit('count/s'))
calibration_flux

In [ ]:
plt.plot(wvl_solution, calibration_flux * calib_final) #sanity check
# plt.xlim(3700, 5000)

### Creating flux calibration

In [ ]:
ref_spectrum_filename = os.path.join(pysyn_env_location, 'grid', 'k93models', 'kp00', 'kp00_10000.fits')

# from https://articles.adsabs.harvard.edu/full/2006MNRAS.368..247C/0000247.000.html: Teff: 9900+-220, log g: 4.00+-0.10
# assuming metallicity [M/H] = 0 and T = 10000K hence, kp00_10000
ref_spectrum = fits.getdata(ref_spectrum_filename)
ref_wave_calib = ref_spectrum['WAVELENGTH']
ref_flux_calib = ref_spectrum['g40']  # at log(g) = 4.0

In [ ]:
plt.plot(ref_wave_calib, ref_flux_calib, label='reference star (HD 158261)')
plt.xlim(3500, 10000)
plt.xlabel(S.Vega.waveunits)
plt.ylabel(S.Vega.fluxunits)
plt.title(os.path.basename(S.Vega.name))
#plt.plot(S.Vega.wave, S.Vega.flux / np.max(S.Vega.flux))

vega_flux_conversion = np.max(ref_flux_calib) / np.max(S.Vega.flux) # scaling vega to reference star
plt.plot(S.Vega.wave, S.Vega.flux * vega_flux_conversion, alpha=0.6, label='Vega')

plt.xlim(3500, 6000)
plt.legend();

The shapes of the two spectra are identical (just different units) -- I ended up using Vega's spectrum because it behaved better when upsampling. The units of the reference spectrum are ergs/s/cm^2/A and are *surface flux*, meaning we do need to do distance scaling.

In [ ]:
import astropy.units as u
from astropy import constants as const
ref_flux = S.Vega.flux * vega_flux_conversion
ref_wave = S.Vega.wave

# convert surface flux to observed flux
d_vega = 25.05 * u.lightyear
r_vega = 164.32 * 10**8 * u.cm
irradiance = ref_flux * u.erg / u.s / u.cm**2 / u.Angstrom

F = irradiance * r_vega.to(u.m)**2 / d_vega.to(u.m) **2
F

In [ ]:
# converting to photon flux now as in PS3 - we don't need to convert because we aren't gonna integrate peaks to find brightness
freqs = const.c / (ref_wave * u.Angstrom).to(u.m)
E = const.h * freqs # J
photon_flux = F / E
photon_flux = photon_flux.to(1 / u.s / u.Angstrom / u.cm**2)  # photons/s/A/cm^2
photon_flux

In [ ]:
plt.plot(ref_wave, photon_flux)
plt.axvline(np.max(wvl_range_B_trunc), color='r', linestyle='--')
plt.axvline(np.min(wvl_range_R), color='b', linestyle='--')
plt.xlim(3500, 10000)
plt.xlabel("Wavelength [A]")
plt.ylabel("Reference star photon flux")

In [ ]:
# upsampling reference spectrum
from scipy import signal
ref_flux_B = photon_flux[(ref_wave > np.min(wvl_range_B_trunc)) & (ref_wave < np.max(wvl_range_B_trunc))]
resampled_flux_B = signal.resample(ref_flux_B, wvl_range_B_trunc.shape[0])

ref_flux_R = photon_flux[(ref_wave > np.min(wvl_range_R)) & (ref_wave < np.max(wvl_range_R))]
resampled_flux_R = signal.resample(ref_flux_R, wvl_range_R.shape[0])

# TODO: need to trim edges because of artifacts created by Fourier-window interpolation
plt.plot(wvl_range_B_trunc, resampled_flux_B)
plt.plot(wvl_range_R, resampled_flux_R)
plt.xlabel("Wavelength (Angstroms)")
plt.ylabel("Resampled reference spectrum (photon/s/A/cm^2)")
# plt.xlim(3500, 6000)

In [ ]:
ref_flux_resampled_combined = np.hstack([resampled_flux_B, resampled_flux_R]) / u.s / u.Angstrom / u.cm**2
ref_flux_resampled_combined *= np.gradient(wvl_solution) * u.Angstrom 
palomar_radius = 5.08 / 2 * u.m

ref_flux_resampled_combined *= (palomar_radius.to(u.cm))**2 * np.pi
plt.plot(wvl_solution, ref_flux_resampled_combined)
plt.xlabel(f'Wavelength [A]')
plt.ylabel(f'Photon Flux [{ref_flux_resampled_combined.unit}]')


## Process Science data
- All above steps on science data to extract spectrum from images

### Rectify, reduce, airmass-correct frames

In [ ]:
science = hdr_df[(hdr_df['IMGTYPE']=='object') & (hdr_df['OBJECT'] != 'HD 158261')]
science

In [ ]:
# TODO: if applying airmass correction, then replace create_master with following function 
# that does median stacking after applying rectification, reduction, correction to each individual frame
def master_science_frame(color):
    # TODO: split red and blue

    science_wvl, extinction_wvl, extinction = None #TODO: add np arrays when available
    science_frames = []
    for fname in science.index:
        frame = util.rectify_frame(data[fname], trace_y_R, y_bound_upper_R, y_bound_lower_R)
        frame = util.reduce_frame(frame, science.loc[fname, 'EXPTIME'], normalised_flat_R, master_bias_red)
        frame = util.correct_airmass(science_wvl, frame, extinction_wvl, extinction, science.loc[fname, 'AIRMASS'])
        science_frames.append(frame)

    return np.median(np.array(science_frames), axis=0)

In [ ]:
science_master_red = create_master(science, 'red', master_bias_red)
science_master_blue = create_master(science, 'blue', master_bias_blue)
science_master_red.shape, science_master_blue.shape

In [ ]:
# apply step #1-2 from "Rectify red" section
science_red_rectified = util.rectify_frame(
    science_master_red, trace_y_R, y_bound_upper_R, y_bound_lower_R
    )[:, x_trim_mask_R]

util.plot_frame2(science_red_rectified, label='counts/s', title='Rectified SN 2024eze (red)', prange=prange_R)
plt.axhline(trace_y_idx_R, color='r', linewidth=0.5)

In [ ]:
# apply step #1-4 from "Rectify blue" section
science_blue_rectified = util.rectify_frame(
    science_master_blue.T, trace_y_B, y_bound_upper_B, y_bound_lower_B
    )[:, x_trim_mask_B][:,::-1]

util.plot_frame2(science_blue_rectified, label='counts/s', title='Rectified SN 2024eze (blue)', prange=prange_R)
plt.axhline(trace_y_idx_B, color='r', linewidth=0.5)

In [ ]:
reduced_rect_science_R = science_red_rectified / normalised_flat_R

util.plot_frame2(reduced_rect_science_R, label='counts/s', title='Reduced, rectified SN 2024eze (red)', prange=prange_R)
plt.axhline(trace_y_idx_R, color='r', linewidth=0.5, alpha=0.7)

In [ ]:
reduced_rect_science_B = science_blue_rectified / normalised_flat_B

util.plot_frame2(reduced_rect_science_B, label='counts/s', title='Reduced, rectified SN 2024eze (blue)', prange=prange_B)
plt.axhline(trace_y_idx_B, color='r', linewidth=0.5, alpha=0.7)

### Extract source and background
#### Red

In [ ]:
sci_aperture_idx_R = (trace_y_idx_R+2-5, trace_y_idx_R+2+6)

plt.plot(np.sum(reduced_rect_science_R, axis=1))
plt.title('Row-summed Red science data')

plt.axvline(trace_y_idx_R+2, color='r', alpha=0.8)
plt.axvline(sci_aperture_idx_R[0], color='r', alpha=0.8)
plt.axvline(sci_aperture_idx_R[1], color='r', alpha=0.8)
plt.xlim(50, 200)

In [ ]:
util.plot_frame(reduced_rect_science_R, prange=(10, 90), aspect=7)
plt.title('Red science data (with source and background apertures)')

plt.axhline(y=sci_aperture_idx_R[0], color='r', alpha=0.5)
plt.axhline(y=sci_aperture_idx_R[1], color='r', alpha=0.5)


sci_bg_upper_idx_R = (sci_aperture_idx_R[1]+5, sci_aperture_idx_R[1]+15)
sci_bg_lower_idx_R = (sci_aperture_idx_R[0]-15, sci_aperture_idx_R[0]-5)
plt.axhline(sci_bg_upper_idx_R[0], color='cyan', alpha=0.8)
plt.axhline(sci_bg_upper_idx_R[1], color='cyan', alpha=0.8)
plt.axhline(sci_bg_lower_idx_R[0], color='yellow', alpha=0.8)
plt.axhline(sci_bg_lower_idx_R[1], color='yellow', alpha=0.8)

plt.ylim(trace_y_idx_R - 60,trace_y_idx_R + 60); 

In [ ]:
sci_source_spec_R = np.sum(reduced_rect_science_R[sci_aperture_idx_R[0] : sci_aperture_idx_R[1]+1], axis=0)

plt.plot(wvl_range_R, sci_source_spec_R)
plt.ylabel('counts/s')
plt.xlabel('Wavelength [A]')
plt.title('SN 2024eze spectrum w/ Background (RED)')

In [ ]:
sci_bg_spec_R = np.median(np.concatenate((
    reduced_rect_science_R[sci_bg_lower_idx_R[0]: sci_bg_lower_idx_R[1]+1],
    reduced_rect_science_R[sci_bg_upper_idx_R[0]: sci_bg_upper_idx_R[1]+1]),
    axis=0), axis=0)

plt.plot(wvl_range_R, sci_bg_spec_R)
plt.ylabel('counts/s')
plt.xlabel('Wavelength [A]')
plt.title('SN 2024eze background (RED)')

In [ ]:
sci_aperture_nrows_R = sci_aperture_idx_R[1] - sci_aperture_idx_R[0]+1
science_spectrum_R = sci_source_spec_R - (sci_bg_spec_R * sci_aperture_nrows_R)

plt.plot(wvl_range_R, science_spectrum_R)
plt.ylabel('counts/s')
plt.xlabel('Wavelength [A]')
plt.title('SN 2024eze background-subtracted spectrum (RED)');

#### Blue

In [ ]:
sci_aperture_idx_B = (trace_y_idx_B-3-6, trace_y_idx_B-3+6)

plt.plot(np.sum(reduced_rect_science_B, axis=1))
plt.title('Row-summed BLUE science data')

plt.axvline(trace_y_idx_B-3, color='r', alpha=0.8)
plt.axvline(sci_aperture_idx_B[0], color='r', alpha=0.8)
plt.axvline(sci_aperture_idx_B[1], color='r', alpha=0.8)
plt.xlim(150, 250)

In [ ]:
util.plot_frame(reduced_rect_science_B, prange=(8, 92), aspect=4)
plt.title('BLUE science data (with source and background apertures)')

plt.axhline(y=sci_aperture_idx_B[0], color='r', alpha=0.5)
plt.axhline(y=sci_aperture_idx_B[1], color='r', alpha=0.5)


sci_bg_upper_idx_B = (sci_aperture_idx_B[1]+5, sci_aperture_idx_B[1]+15)
sci_bg_lower_idx_B = (sci_aperture_idx_B[0]-15, sci_aperture_idx_B[0]-5)
plt.axhline(sci_bg_upper_idx_B[0], color='cyan', alpha=0.8)
plt.axhline(sci_bg_upper_idx_B[1], color='cyan', alpha=0.8)
plt.axhline(sci_bg_lower_idx_B[0], color='yellow', alpha=0.8)
plt.axhline(sci_bg_lower_idx_B[1], color='yellow', alpha=0.8)

plt.ylim(trace_y_idx_B - 60,trace_y_idx_B + 60); 

In [ ]:
sci_source_spec_B = np.sum(
    reduced_rect_science_B[sci_aperture_idx_B[0] : sci_aperture_idx_B[1]+1, :end_idx_B], 
    axis=0)

plt.plot(wvl_range_B_trunc, sci_source_spec_B)
plt.ylabel('counts/s')
plt.xlabel('Wavelength [A]')
plt.title('SN 2024eze spectrum w/ Background (BLUE)')

In [ ]:
sci_bg_spec_B = np.median(np.concatenate((
    reduced_rect_science_B[sci_bg_lower_idx_B[0]: sci_bg_lower_idx_B[1]+1],
    reduced_rect_science_B[sci_bg_upper_idx_B[0]: sci_bg_upper_idx_B[1]+1]),
    axis=0)[:, :end_idx_B], axis=0)

plt.plot(wvl_range_B_trunc, sci_bg_spec_B)
plt.ylabel('counts/s')
plt.xlabel('Wavelength [A]')
plt.title('SN 2024eze background (BLUE)')

In [ ]:
sci_aperture_nrows_B = sci_aperture_idx_B[1] - sci_aperture_idx_B[0]+1
science_spectrum_B = sci_source_spec_B - (sci_bg_spec_B * sci_aperture_nrows_B)

plt.plot(wvl_range_B_trunc, science_spectrum_B)
plt.ylabel('counts/s')
plt.xlabel('Wavelength [A]')
plt.title('SN 2024eze background-subtracted spectrum (BLUE)');

### Calibrate combined spectrum

In [ ]:
science_spectrum_combined = np.hstack([science_spectrum_B, science_spectrum_R]) * u.Unit('count/s')
plt.plot(wvl_solution, science_spectrum_combined)
plt.xlabel("Wavelength [A]")
plt.ylabel("counts / s")
plt.title("SN 2024eze spectrum (COMBINED)");

In [ ]:
science_spectrum_calibrated = science_spectrum_combined * calibration_flux
science_spectrum_calibrated

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(wvl_solution, science_spectrum_calibrated)

plt.ylim(0, 0.4e-15)
plt.xlabel("Wavelength [A]")
plt.ylabel("Flux [$erg\ s^{-1}\ cm^{-2}\ A^{-1}$]")
plt.title("SN 2024eze calibrated spectrum");

## Data Analysis

In [ ]:
tns_data = pd.read_csv('tns_2024eze_2024-04-12_11-13-24_P200_DBSP_ZTF.ascii',
                        sep='\t',
                        na_values=['None'],
                        names=['wavelength', 'flux'],
                        dtype={'wavelength': 'float64', 'flux': 'float64'})
tns_data

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(wvl_solution, science_spectrum_calibrated/np.max(science_spectrum_calibrated), 
         label='2024-05-04 (observed)')
plt.plot(tns_data.wavelength, tns_data.flux/np.max(tns_data.flux), 
         label='2024-04-12 (TNS)', alpha=0.5)

# plt.ylim(0, 0.4e-15)
plt.xlabel("Wavelength [A]")
plt.ylabel("Normalised flux")
plt.title("SN 2024eze spectral comparison")
plt.legend();